# Reading parquet the hard way

Parquet is a cloud-native data format suitable for a variety of data types, including geospatial vector data. It is often described as being "column-oriented", as opposed to row-oriented like traditional databases and vector formats (though in reality parquet is really more of a hybrid format, one not strictly columnar).

The cloud-native aspect of parquet mainly comes down to two facets of its design:

* file metadata is consolidated at the end of a file, and can be read entirely in at most two requests
* the metadata contains enough information for clients to selectively read only chunks that may contain targeted rows

## Predicate pushdown

This "selective reading" is done by a process called "predicate pushdown". Predicate pushdown enables clients to skip reading potentially massive amounts of data by leveraging things like chunk statistics and chunk bloom filters to restrict reads to only those chunks that _may_ potentially contain target data.

Consider an example like this SQL statement:

```sql
SELECT temperature
FROM dataset
WHERE
  st_intersects(geom, AOI)
  AND snow_cover_percent > 0
  AND precipitation >= 10
  AND land_cover NOT IN ("rock", "urban")
```

Here we see the user requesting the `temperature` variable from `dataset` records that have:

* a geometry interseting the AOI
* any snow cover
* some significant precipitation
* a non-impervious surface

Each of these conditions is called a predicate. Predicate pushdown means using predicates to filter what chunks to read from a file, or to even filter out whole files. File and chunk statistics provide a means of identifying files and chunks that cannot possibly have a record of interest per the query predicates, and skipping those. Of those remaining, bloom filters can provide another efficient check to eliminate chunks that cannot possibly contain target records. What chunks are left are then read and filtered by the client application using the actual values prior to returning the requested data.

Note the language "cannot possibly contain" is used here specifically: we never want to miss a matching record, so we use techniques that give us means of eliminating chunks with certainty that they will not contain a recod of interest. These techniques come with the trade off that false positives can occur: we can rarely know that chunk _does_ have a target record. Sometimes we then read chunks without target records, because that is the only way to ensure they do not have what we are looking for.

Generally though, predicate pushdown is an effective means of eliminating a large amount of data without having to read anything but metadata.

## The exercise

To get a deeper understanding of parquet and how it works, we're going to examine the [Overture Maps Buildings dataset](https://docs.overturemaps.org/guides/buildings). We will find the geometry for the building we're in for this workshop, the AUT School of Business building.

This exercise is perhaps a bit contrived, because we'll start with a rough geometry for the building that we traced from aerial imagery in exercise 1, but the principles we'll be demonstrating are applicable generally for clients wanting to read from parquet, and will show how that process works in some detail. Key to the process will be a focus on using predicate pushdown with geometries, and how we can use less expensive bounding box operations to more effiicently eliminate chunks and records before turning to more expensive real-geometry comparisons.

### The `por-que` library

To facilitate the learning experience, we'll use the author's custom python parquet parsing library, called `por-que`. This library is different from other, more efficient parsing libraries like `pyarrow`, in that it is designed for the educational experience. A core feature is the parsing of the entire file metadata, data page metadata, and byte structure of the file, which it exposes to users, unlike other tools that keep them hidden away as internal details.

`por-que` can export the entire parsed structure to JSON, which can then be loaded, visualized, and explored with the [`ver-por-que` web application](https://teotl.dev/ver-por-que/). We'll see all of this in action through the exercise.

But first, let's get some prep out of the way, starting with the necessary imports.

In [ ]:
import asyncio
import json

from contextlib import AsyncExitStack
from dataclasses import dataclass
from pathlib import Path
from typing import Self
 
import s3fs

from devtools import pprint
from por_que import AsyncHttpFile, FileMetadata, ParquetFile
from shapely import (
    intersection,
    from_geojson as shapely_from_geojson,
    from_wkb as shapely_from_wkb,
)

We'll also define a helper function for zipping async iterators together, which we'll use later.

In [ ]:
async def azip(*iterators):
    """This is a silly zip implementation for async iterators that
    assumes equal length. Don't copy this. It's not robust."""
    _iter = iter(iterators)
    first, rest = next(_iter), list(_iter)
    async for val in first:
        yield (val, *[await anext(iterator) for iterator in rest])

And we'll define a bounding box model, which we will be able to use for all of the bounding box operations in this exercise. Note that we could have used a existing model from a library, but by implementing our own we can see how simple the bound box operations are.

In [ ]:
@dataclass(slots=True, frozen=True)
class BBox:
    xmin: int | float
    ymin: int | float
    xmax: int | float
    ymax: int | float

    def intersects(self, other: Self) -> bool:
        return (
            other.xmin < self.xmax
            and self.xmin < other.xmax
            and other.ymin < self.ymax
            and self.ymin < other.ymax
        )

    @classmethod
    def from_linestring(cls, linestring: list[list[int | float]]) -> Self:
        xmin = float('inf')
        ymin = float('inf')
        xmax = float('-inf')
        ymax = float('-inf')

        try:
            for x, y in linestring:
                xmin = min(xmin, x)
                ymin = min(ymin, y)
                xmax = max(xmax, x)
                ymax = max(ymax, y)
        except Exception:
            raise ValueError('Failed to extract bbox')

        return cls(xmin, ymin, xmax, ymax)

Lastly, we'll define the geometry we'll be using for this exercise, and we'll use it with our `BBox` class to construct its bounding box. Again, this is the same polygon we delineated in exercise 1.

In [ ]:
geom = json.loads('''{
  "coordinates": [
    [
      [
        174.76536299052356,
        -36.85325730119731
      ],
      [
        174.76501948066357,
        -36.85354934760823
      ],
      [
        174.76510987799577,
        -36.853728372411425
      ],
      [
        174.76557768418712,
        -36.85354844344181
      ],
      [
        174.76544321815658,
        -36.85331878474462
      ],
      [
        174.76536299052356,
        -36.85325730119731
      ]
    ]
  ],
  "type": "Polygon"
}''')
geom

In [ ]:
geom_bbox = BBox.from_linestring(geom['coordinates'][0])
geom_bbox

## Overture Maps parquet data

Overture Maps has a number of datasets available in parquet format, including the building footprints dataset we'll be using for this exercise. All are distributed from both AWS and Azure using their respective object storage services. We'll use the files hosted on S3; we can get the S3 url prefix from [the buildings dataset documentation](https://docs.overturemaps.org/guides/buildings).

In [ ]:
s3_url_prefix = 's3://overturemaps-us-west-2/release/2025-09-24.0/theme=buildings/type=building/'

### Listing the files

With an S3 prefix, we can use `s3fs` to make a "filesystem" instance with which we can see all the files in the dataset.

In [ ]:
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
parquet_parts = fs.ls(s3_url_prefix)
parquet_parts

In [ ]:
len(parquet_parts)

Wow, that's a lot of files! But it is not unexpected: this is a worldwide building footprints dataset, so it has _a lot_ of records (we'll see exactly how many in a bit). To make it more efficient to access a subset of the dataset, Overture partitions the rows into files around 1 GB in size. So we have a ton of data here, how will we ever find the record for which we are searching?

To enable the next step, we need these as HTTPS URLs. Let's transform them.

In [ ]:
parquet_urls = [
    f'https://{bucket}.s3.{"-".join(bucket.split('-')[1:])}.amazonaws.com/{key}'
    for bucket, key in map(lambda x: x.split('/', 1), parquet_parts)
]
parquet_urls

## Loading one of these files using `por-que`

`por-que` exposes three key classes, each of which we'll be using throughout the rest of this exercise:

* `AsyncHttpFile`
* `FileMetadata`
* `ParquetFile`

The first of these, `AsyncHttpFile`, is a helper class that can open an HTTP(S) URL and expose it as a readable filelike object. We need this to allow the other two classes to do their parsing of the parquet files over HTTP.

`FileMetadata` is a class that represents the block of metadata at the end of a parquet file. This metadata includes both file level information, like the schema, and chunk level metadata via row groups and their column chunks. Passing a readable filelike object into `FileMetadata.from_reader()` will parse the metadata object from said file.

`ParquetFile` is a class that represents then entire physical and logical structure of a parquet file. This includes `FileMetadata`, but also all data page locations and metadata. `ParquetFile` and its nested `DataPage` objects allow reading data from the file. `ParquetFile.from_reader()`, similar to above, will parse the full structure and metadata from a passed in readable filelike object.

Let's see how we can use `AsyncHttpFile` with `ParquetFile` and some of what the latter exposes. Note that creating the `ParquetFile` instances has to make many random reads within the file, so this is not a fast process, especially over the network (typically taking around 30 seconds).

In [ ]:
async with AsyncHttpFile(parquet_urls[0]) as hf:
    pf = await ParquetFile.from_reader(hf, parquet_urls[0])

In [ ]:
pprint(pf.metadata.schema_root)

In [ ]:
pprint(pf.metadata.row_groups[0])

### Visualizing the `ParquetFile` structure

We can dump our `ParquetFile` model instance to JSON, and then we can load that into a static web application called [ver-por-que](https://teotl.dev/ver-por-que). The app will show the structure via an interactive visualization.

So let's first write our JSON file:

In [ ]:
Path('./buildings.parquet.json').write_text(pf.to_json())

Now, download the json file (right-click on the file in the file explorer sidebar), then browse to [ver-por-que](https://teotl.dev/ver-por-que) and load the JSON file.

Key things to notice:

* how data page region is logically organized into row groups and column chunks, but all the row group and column chunk information comes from the file metadata
* the schema is not a flat set of columns: it has nested data structures, like the bbox columns
* the general layout of the file bytes

### File-level geo metadata

Another thing to note is that the parquet specification does not have file-level statistics or bloom filters--such information is available (optionally) at the column chunk (and sometimes data page) level, but not the file level.

To faciliate more performant reading by allowing filtering at the file level, convention dictates writing geospatial metadata to the top-level of the file using the flexible "key/value" metadata in the file metadata structure. This geospatial metadata is written in JSON format, and includes the bounding box around all geometries in th file.

We can view that metadata like so:

In [ ]:
json.loads([kv for kv in pf.metadata.key_value_metadata if kv.key == 'geo'][0].value)

## Filtering files

Now that we know how to determine the bounding box for a file's geometries, we can take a first pass at eliminating data we'll need to read by finding only those files that have a bounding box that intersects our geometry's bounding box.

### Reading the metadata for all files

Above, we read the geospatial metadata for a file, but we need to do it for all files. Reading the metadata can be slow due to network round trips and latency. To make reading the metadata across all files possible in a somewhat reasonable amount of time we can leverage parallelism, but that comes at the cost of code complexity.

In essence, the code below is creating an `AsyncHttpFile` instance for every file in our list. We open each of those instances within a context manager, so they will be automatically closed once we leave this context (because each consumes a not-insignificant amount of memory while they remain open). With each of those `AsyncHttpFile` instances we can create a corresponding `FileMetadata` instance via an async function call. We put all those async calls into a list, so we can use `asyncio.gather` to run them in parallel, and we `await` the completion of the gather to get all the produced `FileMetadata` instances. Again, parallelism allows us to make many HTTP requests for the data at once, instead of having to wait as we would when running requests serially.

These `FileMetadata` instances we zip up with the corresponding URLs, and we use the resulting tuples to construct a dictionary keyed on the file URL. This gives us a mapping of a file's URL to its `FileMetadata` instance.

We'll also collect and output the total number of bytes in this dataset summed across all files.

**Note**: this process is rather slow. We have to download ~300 MB, parse and instantiate all of the metadata data structures. In testing this took anywhere from 2 to 5 minutes... If this takes too long, feel free to skip this part by reviewing the outputs in the completed notebook, and copy the `urls_that_intersect` values and define that before proceeding to steps that need those URLs. 

In [ ]:
fm_tasks = []
total_bytes = 0
async with AsyncExitStack() as stack:
    for url in parquet_urls:
        f = await stack.enter_async_context(AsyncHttpFile(url))
        total_bytes += f.size
        fm_tasks.append(asyncio.create_task(FileMetadata.from_reader(f)))
    fms = dict(zip(parquet_urls, await asyncio.gather(*fm_tasks)))

print(f'{total_bytes:_}')

Let's also see how many rows we have in this dataset across all the files.

In [ ]:
total_rows = 0
for fm in fms.values():
    total_rows += fm.row_count

print(f'{total_rows:_}')

### Finding intersecting files by their bounding boxes

We can iterate through our file URLs and their `FileMetadata` instances to build a `BBox` for each from the geosptial metadata. We do the same `json.loads()` call on the key/value metadata entry with the `geo` key that we saw above; from the resulting dictionary we can build a `BBox` from the `['columns']['geometry']['bbox']` fields.

Once we have a file's `BBox`, we can check for an intersection with out geometry's `BBox` and note where we find one. At the end we will have a list of the URLs for all files that are worthy of a closer look. Any that get filtered out we can know with certainty do not contain rows of interest, so we can safely disregard them.

In [ ]:
urls_that_intersect = []
for fm_url, fm in fms.items():
    kv_metadata = json.loads(
        [
            kv for kv in fm.key_value_metadata
            if kv.key == 'geo'
        ][0].value,
    )
    bbox = BBox(*kv_metadata['columns']['geometry']['bbox'])
    if bbox.intersects(geom_bbox):
        print(bbox)
        urls_that_intersect.append(fm_url)

print(geom_bbox)
urls_that_intersect

**If you skipped this section because it was too slow, define the `urls_that_intersect` array here before proceeding.**

## Finding intersecting row groups

We've successfully filtered down the file set to a much smaller number, but we can still filter the data we need to read even further. To do so, we can perform the same bounding box filtering we did at the file level, but at the row group level using the column chunk statistics across our bounding box columns.

Let's walk through this process for one file, and once we see how that process comes together we can create a single routine to perform the checks across each file in our filtered set.

To begin, we'll grab the `FileMetadata` instance for one of our matching URLs. From that we can start looking into the logical structures of the file, the row groups and column chunks.

In [ ]:
fm = fms[urls_that_intersect[0]]

In [ ]:
bbox_min_cc0 = fm.row_groups[0].column_chunks['bbox.xmin']
pprint(bbox_min_cc0)

We see the structure of the file starting to take shape:

* Files have rows split up into row groups
* Files have columns, but these are not accessed directly
* Row groups have a column chunk for each column in the file

Parquet is not truly column-oriented! It is column oriented within row groups, but within a file the organization is something of a hybrid between column and row-orientation.

What does this mean for us? We cannot just read a column. But that's okay, because if we identify column chunks that might contain data we're looking for then we can identify what row groups definitely don't have records we want, and we have a means of filtering down the data we need to read.

But how do we do this? We can use the column chunk statistics. Specifically the min and max values.

In [ ]:
bbox_min_cc0.statistics.min_value

In [ ]:
bbox_min_cc0.statistics.max_value

These values aren't super helpful looking, are they? What do they even mean?

To understand how to interpret these raw bytes, we need to do two things:

* Convert them to the column's physical type
* Convert the physical type value to the column's logical type

To get the type information for a column, we can use the `FileMetadata`'s schema. To make it easier to access the specific schema element for this column, each column chunk has a link to it's schema element in its metadata.

In [ ]:
schema = bbox_min_cc0.metadata.schema_element
schema

We can explicitly get the physical and logical types from this schema element (which are also contained in its string representation above):

In [ ]:
schema.type

In [ ]:
schema.get_logical_type()

In this case, we get a fairly expected answer: the physical type of bounding box coordinates is float, and no additional logical type conversion need happen to use the float values (logical types are super useful in other cases, like telling us a bytes physical type column is a geometry, or encoding other special types like decimals, datetimes, JSON, etc.).

With this type information we can convert our raw bytes values into something more useful. In fact, the schema element has conversion helper methods on it for just this purpose:

In [ ]:
schema.physical_to_logical_type(schema.bytes_to_physical_type(bbox_min_cc0.statistics.max_value))

What's more, the statistics object itself can convert the min and max values for us (using these same convenience methods under the covers):

In [ ]:
{
    'min_value': bbox_min_cc0.statistics.converted_min_value,
    'max_value': bbox_min_cc0.statistics.converted_max_value,
}

Now that we can get this min/max statistics in a useful form, we can build up a bounding box for each row group. Let's make a function to construct a `BBox` instance for a given row group:

In [ ]:
def get_rg_bbox(row_group) -> BBox:
    return BBox(
        row_group.column_chunks['bbox.xmin'].statistics.converted_min_value,
        row_group.column_chunks['bbox.ymin'].statistics.converted_min_value,
        row_group.column_chunks['bbox.xmax'].statistics.converted_max_value,
        row_group.column_chunks['bbox.ymax'].statistics.converted_max_value,
    )

We can try it out and see what we get:

In [ ]:
get_rg_bbox(fm.row_groups[0])

Cool, that works! Let's run it on every row group of our `FileMetadata` instance and check for any intersections

In [ ]:
intersecting_rgs = [rg for rg in fm.row_groups if get_rg_bbox(rg).intersects(geom_bbox)]
len(intersecting_rgs)

Look at that! We've successfully filtered down the rows we need to be concered about significantly, _and we haven't even read any data yet_.

Can we keep using metadata to filter further? Like, does the geometry column have metadata that could be used in a similar manner?

In [ ]:
pprint(intersecting_rgs[0].column_chunks['geometry'])

Hmm, that's not super helpful looking. It might be time to say we've done as much as we can without reading data, and start fetching rows.

## Reading and filtering rows

To read rows we need to get full `ParquetFile` instances for our intersecting files. Once we have a `ParquetFile` instance, we can read its column chunks.

We're going to need a `ParquetFile` instance per intersecting file. Because instantiating them is a time consuming operation, let's instantiate them all in parallel here, building them up into a dictionary keyed on the file URL. Then we can just grab one of them to use for the rest of this section as we prove out our process (making sure it is for the same file as the `FileMetadata` instance we were using above).

In [ ]:
pf_tasks = []
async with AsyncExitStack() as stack:
    for url in urls_that_intersect:
        f = await stack.enter_async_context(AsyncHttpFile(url))
        pf_tasks.append(asyncio.create_task(ParquetFile.from_reader(f, url)))
    pfs = dict(zip(urls_that_intersect, await asyncio.gather(*pf_tasks)))

pf = pfs[urls_that_intersect[0]]

With a `ParquetFile` instance, we can find all four of the `bbox` column chunks in one of our intersected row groups. We do this by iterating through all column chunks in the file, keeping those that are part of our intersected row group index (given by that row group's `ordianal` property) that have a schema path starting with `bbox`. The column chunks we collect here we'll put into a dictionary, keyed on the column chunk path in the schema (e.g., `bbox.min`).

In [ ]:
bbox_chunks = {
    cc.path_in_schema: cc
    for cc in pf.column_chunks
    if cc.row_group == intersecting_rgs[0].ordinal and cc.path_in_schema.startswith('bbox')
}
bbox_chunks.keys()

Now that we have identified the relevant column chunks and inspected what they are, let's take a closer look at one to see other metadata it has.

In [ ]:
pprint(bbox_chunks['bbox.xmin'])

We see from the above that the name of the type for our column chunk instances is `PhysicalColumnChunk`. That's because these column chunks are not the column chunk metadata in the file metadata, but are instead representative of all the physical byte ranges in the file that make up this "column chunk" abstraction within the file. This includes the column chunk metadata from the file metadata, any indices in the file metadata (column index or offset index), any dictionary page, or, most prominently, the data pages that store the data for the column chunk.

As a result, this `PhysicalColumnChunk` type has a method `parse_all_data_pages()`, which we can use read all data pages in the column chunk and parse the data values from them. A dictionary page, if present, will also be read so the data pages can be correctly decompressed.

Let's use `parse_all_data_pages()` with each of our bbox chunks to get all the bbox values for our target row group. We have to have a readable filetype object to pass in because we need to read file data, so we'll use the `AsyncHttpFile` class again and pass in an open instance of that class. The data values we read we'll zip together into a four-tuple like `(xmin, ymin, xmax, ymax)`; this tuple provides us a data structure we can use to instantiate `BBox` instances for each row in a later step.

In [ ]:
async with AsyncHttpFile(urls_that_intersect[0]) as hf:
    bbox_tuples = [bbox_tuple async for bbox_tuple in azip(
        bbox_chunks['bbox.xmin'].parse_all_data_pages(hf),
        bbox_chunks['bbox.ymin'].parse_all_data_pages(hf),
        bbox_chunks['bbox.xmax'].parse_all_data_pages(hf),
        bbox_chunks['bbox.ymax'].parse_all_data_pages(hf),
    )]

Let's take a look at what the first four rows look like.

In [ ]:
bbox_tuples[:4]

This result, it might be unexpected. Each data value we see is a three-tuple like `(float, int, int)`. The float values, if not obviously, are our actual data values. The integer values are the definition and repetition levels for each value.

Exactly what the definition and repetition levels are and how they work is outside the scope of this exercise; the short version is they are used in reconstructing nested types like maps, arrays, and structs, by providing the necessary state to determine when and where within a data tree to end/start a data structure or insert nulls. To learn more about how this works, review the three-part blog post series on the Apache Arrow blog ([part 1](https://arrow.apache.org/blog/2022/10/05/arrow-parquet-encoding-part-1/), [part 2](https://arrow.apache.org/blog/2022/10/08/arrow-parquet-encoding-part-2/), and [part 3](https://arrow.apache.org/blog/2022/10/17/arrow-parquet-encoding-part-3/)) or dig into the `por_que.structuring` code.

For now, just note that for our purposes we only need the first element of these tuples, which we can get using something like the generator expression `(i[0] for i in bbox_tuple)`. Let's use that when iterating through each of our bbox tuples, constructing a `BBox` instance from each tuple's extracted values and checking the intersection with our geometry's `BBox` instance. We'll print out the index of each row that intersects, if any.

In [ ]:
for row_index, bbox_tuple in enumerate(bbox_tuples):
    if BBox(*(i[0] for i in bbox_tuple)).intersects(geom_bbox):
        print(row_index)

Now we have all the steps we need to identify rows that intersect our search bounding box. Let's put all these steps together so we can iterate through all the intersecting URLs and find any and all rows with intersecting bounding boxes!

## Putting this all together

Let's make a function that, given a `ParquetFile` instance, will search for intersecting row group bounding boxes, then search those row groups for rows with intersecting bounding boxes.

In [ ]:
async def find_intersecting_rows(pf: ParquetFile) -> dict[int, list[int]]:
    matched_rows = {}
    for rg in pf.metadata.row_groups:
        if not get_rg_bbox(rg).intersects(geom_bbox):
            continue

        bbox_chunks = {
            cc.path_in_schema: cc
            for cc in pf.column_chunks
            if cc.row_group == rg.ordinal and cc.path_in_schema.startswith('bbox')
        }

        # we should be able to use the same AsyncHttpFile but
        # some as-of-yet-not-understood issue is causing a
        # ServerDisconnectedError when we reuse
        async with AsyncHttpFile(pf.source) as hf:
            bbox_tuples = [bbox_tuple async for bbox_tuple in azip(
                bbox_chunks['bbox.xmin'].parse_all_data_pages(hf),
                bbox_chunks['bbox.ymin'].parse_all_data_pages(hf),
                bbox_chunks['bbox.xmax'].parse_all_data_pages(hf),
                bbox_chunks['bbox.ymax'].parse_all_data_pages(hf),
            )]
        
        for row_index, bbox_tuple in enumerate(bbox_tuples):
            if BBox(*(i[0] for i in bbox_tuple)).intersects(geom_bbox):
                try:
                    matched_rows[rg.ordinal].append(row_index)
                except KeyError:
                    matched_rows[rg.ordinal] = [row_index]

    return matched_rows

Now let's use our `find_intersecting_rows` function on each of our intersecting `ParquetFile` instances. We'll track the outputs in a nested dictionary structure mapping file URL to row group index to intersected row indices.

In [ ]:
matched_rows: dict[str, dict[int, list[int]]] = dict(
    zip(
        urls_that_intersect,
        await asyncio.gather(*[
            asyncio.create_task(find_intersecting_rows(pf))
            for pf in pfs.values()
        ])
    )
)
matched_rows

As only one row group in one file had intersections, let's extract the key bits of information here out into discrete variables, to keep the following code simpler. Specifically, we'll define vars for the file URL, the row group index, and the intersected row indices.

In [ ]:
file_url, row_group_index, row_indices = [(url, k, v) for url, d in matched_rows.items() if d for k, v in d.items()][0]
file_url, row_group_index, row_indices

## Reading data for the intersecting rows

### A simple string column

We can use the row group index and row indices to read other column chunks that contain other data for the rows. Let's take a look at how that works with the `names.primary` column, to see if that provides any insight into the intersected rows.

To do this, we need to first find the column chunk for this `name.primary` column that is within our target row group. We can iterate through the target `ParquetFile`'s column chunks until we find the one that matches.

In [ ]:
for cc in pfs[file_url].column_chunks:
    if cc.row_group == row_group_index and cc.path_in_schema == 'names.primary':
        primary_name_chunk = cc
        break

pprint(primary_name_chunk)

With the column chunk identified, we can parse all its data pages to get all its rows.

In [ ]:
async with AsyncHttpFile(file_url) as hf:
    name_rows = [name async for name in primary_name_chunk.parse_all_data_pages(hf)]

Now it's simply a matter of grabbing each row with our target row indices.

In [ ]:
for row in row_indices:
    print(name_rows[row])

Easy yeah?

### Reading the geometries

We can do this same process again to get access to the target row geometries. So let's do that, starting again by finding the column chunk for the `geometry` column in our target row group.

In [ ]:
for cc in pfs[file_url].column_chunks:
    if cc.row_group == row_group_index and cc.path_in_schema == 'geometry':
        geom_chunk = cc
        break

pprint(geom_chunk)

Now we can read the rows from the column chunk. This time we'll combine that operation with the row filtering as well, so we just collect our target geometries. Let's also print those out and see what they look like.

In [ ]:
async with AsyncHttpFile(file_url) as hf:
    geom_rows = [[geom async for geom in geom_chunk.parse_all_data_pages(hf)][i] for i in row_indices]
geom_rows

That doesn't look too useful, does it?

The type for this column is `BYTE_ARRAY`, and we have no logical type specified in the schema for this column. The addition of parquet `GEOMETRY` and `GEOGRAPHY` logical types is [really fairly recent](https://github.com/apache/parquet-format/pull/240). It appears this file does not use either of those types, but instead relies on the older geoparquet specfication that defines the convention of encoding geometries in `BYTE_ARRAY` format with no logical type.

In fact, if we go back to the file metadata and the geospatial key/value metadata, we can see where this encoding was specified:

In [ ]:
json.loads([kv for kv in pfs[file_url].metadata.key_value_metadata if kv.key == 'geo'][0].value)

The `geometry` column we see here is encoded as WKB! Let's take a second look at the values in hex format and see if WKB seems a reasonable interpretation of these byte values. We have to extract just the value from our `(value, definition, repetition)` tuples though, so we'll do that first.

In [ ]:
geoms = [row[0] for row in geom_rows]
for _geom in geoms:
    print(_geom.hex())

What do you think? Do these look like they could be WKB values?

We can use the `from_wkb()` function from shapely to parse these values into geometries, if indeed they are valid. That function is imported here as `shapely_from_wkb()`; let's try it out! We'll collect all the resulting shapes in a dictionary keyed on the row index.

In [ ]:
shapely_geoms = {}
for index, _geom in zip(row_indices, geoms):
    shapely_geom = shapely_from_wkb(_geom)
    shapely_geoms[index] = shapely_geom
    display(shapely_geom)

Hmm, interesting. It may or may not be obvious which of these is our building of interest, but either way let's treat this as a problem to solve robustly. We've done really inexpensive bounding box comparisons up until now to filter down the set of possibilities as cheaply as possible. Now that we have some actual contenders, we should switch to th more expensive comparison of the geometries themselves.

To facilitate this comparison, let's also load our geometry into shapely.

In [ ]:
our_geom = shapely_from_geojson(json.dumps(geom))
display(our_geom)

With shapely geometries on either side, we can find all geometries that intersect our geometry.

In [ ]:
for _geom in shapely_geoms.values():
    if _geom.intersects(our_geom):
        display(_geom)

Hmm, well, we've narrowed it down. But still, we need to do better. How can we estimate similarity? Perhaps if we calculate the actual intersection as a geometry, we could measure its area? The larger the intersecting area, the more likely the geometry is our target building? After all, we might expect some overlaps around the edges that could cause intersecting slivers, but we should not have buildings overlapping in the Overture dataset.

So let's try that out and see what happens.

In [ ]:
max_intersection_area = 0
max_intersecting_geom = None
max_intersecting_index = None

for index, _geom in shapely_geoms.items():
    area = intersection(_geom, our_geom).area
    if area > max_intersection_area:
        max_intersection_area = area
        max_intersecting_geom = _geom
        max_intersecting_index = index

display(max_intersecting_geom)
print(f'The row that most intersects our geometry is {max_intersecting_index} of row group {row_group_index} in {file_url}')

Look at that! We've narrowed it down to a single row. Does this look like it is the right row?

## Questions

* How effective was parquet's support for predicate pushdown in our search? Can the access efficiency be quantified for this particular scenario?
* Using what you learned through this exercise, can you classify what types of problems parquet supports well?
* What about what parquet would be bad at? What are the trade-offs of parquet? What might be done to mitigate its trade-offs? What cannot be mitigated?
* Is parquet a true cloud-native format? Some argue that it is not. Can you make a case for why it is? What about why it might not be?
* What would it take to write a generic client for this access pattern? What about others? What would it take to make a truly general purpose query engine for parquet?